In [1]:
import pandas as pd
import numpy as np
import random
import os
import torch
import sys
sys.path.append('../')
from lillian_modules import anemia_utils, anemia_constants
from lillian_modules.anemia_env import AnemiaEnv
from stable_baselines3.dqn import DQN
# from modules.env import LupusEnv
from sklearn.model_selection import train_test_split
# from stable_baselines3 import DQN
from imblearn.under_sampling import ClusterCentroids, RandomUnderSampler
import warnings
warnings.filterwarnings('ignore')

In [2]:
SEED = anemia_constants.SEED
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

In [3]:
testing_df = pd.read_csv('../../RL-Agent-Diagnosis/synthetic_data/final/data/with_new_stable_baselines/testing_set.csv')
# testing_df = pd.read_csv('../../RL-Agent-Diagnosis/lupus/data/inconclusive_diagnosis/missingness/0/testing_set.csv')
X_test, y_test = testing_df.iloc[:, 0:-1], testing_df.iloc[:, -1]
X_test, y_test = np.array(X_test), np.array(y_test)
testing_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,7.116363,-1.000000,3.781573,2.738413,-1.000000,95.904198,68.457895,2.226085,0,1.892912,39.808550,110.329197,64.404350,21.654404,73.787009,21.349089,-1.000000,5
1,8.125320,92.230003,4.231419,1.188039,143.365567,104.057204,204.747831,2.342554,0,0.652614,13.478089,-1.000000,32.705481,-1.000000,43.520272,24.375961,142.815207,1
2,11.309450,38.324563,-1.000000,-1.000000,455.077909,76.402602,-1.000000,4.440732,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,33.928350,-1.000000,4
3,13.763858,253.513394,2.262606,0.551444,453.772884,82.781943,90.101466,4.987993,0,0.853521,104.005514,34.639227,0.963866,22.083012,88.891838,41.291574,19.856071,0
4,11.464002,-1.000000,-1.000000,-1.000000,320.964653,104.287127,-1.000000,3.297819,0,1.163516,121.616315,105.895897,-1.000000,9.337462,-1.000000,34.392007,-1.000000,7


In [4]:
model = anemia_utils.load_dqn3('../models/anemia_models/basic/best_acc_model.zip')
# model = utils.load_dqn3('../models/inconclusive_diagnosis/04_03_23/23_ft_incorr_-1_corr_1/best_acc_model.zip')
# model = utils.load_dqn3('../models/very_simple_models/best_models/23_ft_20000000/best_acc_model.zip')

Using stable baselines 3
setting up model


In [6]:
test_df = pd.DataFrame()
testing_env = AnemiaEnv(X_test, y_test, random=False)
# testing_env = LupusEnv(X_test, y_test, random=False)
count = 0
try:
    while True:
        count += 1
        if count %(len(X_test)/5)==0:
            print(f'Count: {count}')
        obs, done = testing_env.reset(), False
        while not done:
            action, states = model.predict(obs, deterministic=True)
            obs, rew, done, info = testing_env.step(action)
            if done==True:
                test_df = test_df.append(info, ignore_index=True)
except StopIteration:
    print('Testing done ....')
test_df.head() 

Count: 2800
Count: 5600
Count: 8400
Count: 11200
Count: 14000
Testing done ....


,index,episode_length,reward,y_pred,y_actual,trajectory,terminated,is_success
0,0.0,5.0,1.0,5.0,5.0,"[hemoglobin, rbc, mcv, ret_count, Hemolytic an...",0.0,1.0
1,1.0,5.0,1.0,1.0,1.0,"[hemoglobin, rbc, mcv, segmented_neutrophils, ...",0.0,1.0
2,2.0,6.0,1.0,4.0,4.0,"[hemoglobin, rbc, ferritin, tibc, hematocrit, ...",0.0,1.0
3,3.0,3.0,1.0,0.0,0.0,"[hemoglobin, hematocrit, No anemia]",0.0,1.0
4,4.0,5.0,-1.0,7.0,7.0,"[hemoglobin, rbc, mcv, segmented_neutrophils, ...",1.0,1.0


In [7]:
test_df.episode_length.min(), test_df.episode_length.max()

(3.0, 8.0)

In [8]:
success_rate, success_df = anemia_utils.success_rate(test_df)
success_rate

97.52142857142857

In [10]:
acc, f1, roc_auc = anemia_utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

(0.9752142857142857, 0.9741244219683337, 0.9861905559548687)

In [12]:
avg_length, avg_return = anemia_utils.get_avg_length_reward(test_df)
avg_length, avg_return

(4.856571428571429, 0.7594285714285715)

In [13]:
test_df.to_csv('../../RL-Agent-Diagnosis/synthetic_data/final/test_dfs/new/test_df_basic.csv', index=False)